In [11]:
import numpy as np
import pandas as pd
import os.path
import sqlite3
import tempfile
from Kosieradzki_Bartosz_assignment_2 import *

Posts = pd.read_csv(r"Posts.csv")
Badges = pd.read_csv(r"Badges.csv")
Comments = pd.read_csv(r"Comments.csv")
Users = pd.read_csv(r"Users.csv")
Votes = pd.read_csv(r"Votes.csv")

baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

res1 = pd.read_sql_query("""
                        SELECT STRFTIME('%Y', CreationDate) AS Year, COUNT(*) AS TotalNumber
                           FROM Posts
                           GROUP BY Year
                        """, conn)
res2 = pd.read_sql_query("""
                                    SELECT Id, DisplayName, SUM(ViewCount) AS TotalViews
                                    FROM Users
                                    JOIN (
                                    SELECT OwnerUserId, ViewCount FROM Posts WHERE PostTypeId = 1
                                    ) AS Questions
                                    ON Users.Id = Questions.OwnerUserId
                                    GROUP BY Id
                                    ORDER BY TotalViews DESC
                                    LIMIT 10
                                    """, conn)
res3 = pd.read_sql_query("""
                                    SELECT Year, Name, MAX((Count * 1.0) / CountTotal) AS MaxPercentage
                                    FROM (
                                    SELECT BadgesNames.Year, BadgesNames.Name, BadgesNames.Count, BadgesYearly.CountTotal
                                    FROM (
                                    SELECT Name, COUNT(*) AS Count, STRFTIME('%Y', Badges.Date) AS Year
                                    FROM Badges
                                    GROUP BY Name, Year
                                    ) AS BadgesNames
                                    JOIN (
                                    SELECT COUNT(*) AS CountTotal, STRFTIME('%Y', Badges.Date) AS Year
                                    FROM Badges
                                    GROUP BY YEAR
                                    ) AS BadgesYearly
                                    ON BadgesNames.Year = BadgesYearly.Year
                                    )
                                    GROUP BY Year
                                    """, conn)
res4 = pd.read_sql_query("""
                                    SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
                                    FROM (
                                    SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
                                    CmtTotScr.CommentsTotalScore
                                    FROM (
                                    SELECT PostId, SUM(Score) AS CommentsTotalScore
                                    FROM Comments
                                    GROUP BY PostId
                                    ) AS CmtTotScr
                                    JOIN Posts ON Posts.Id = CmtTotScr.PostId
                                    WHERE Posts.PostTypeId=1
                                    ) AS PostsBestComments
                                    JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
                                    ORDER BY CommentsTotalScore DESC LIMIT 10
                                    """, conn)
res5 = pd.read_sql_query("""
                                        SELECT Posts.Title, STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date, VotesByAge.*
                                        FROM Posts
                                        JOIN (
                                        SELECT PostId,
                                        MAX(CASE WHEN VoteDate = 'before' THEN Total ELSE 0 END) BeforeCOVIDVotes,
                                        MAX(CASE WHEN VoteDate = 'during' THEN Total ELSE 0 END) DuringCOVIDVotes,
                                        MAX(CASE WHEN VoteDate = 'after' THEN Total ELSE 0 END) AfterCOVIDVotes,
                                        SUM(Total) AS Votes
                                        FROM (
                                        SELECT PostId,
                                        CASE STRFTIME('%Y', CreationDate)
                                        WHEN '2022' THEN 'after'
                                        WHEN '2021' THEN 'during'
                                        WHEN '2020' THEN 'during'
                                        WHEN '2019' THEN 'during'
                                        ELSE 'before'
                                        END VoteDate, COUNT(*) AS Total
                                        FROM Votes
                                        WHERE VoteTypeId IN (3, 4, 12)
                                        GROUP BY PostId, VoteDate
                                        ) AS VotesDates
                                        GROUP BY VotesDates.PostId
                                        ) AS VotesByAge ON Posts.Id = VotesByAge.PostId
                                        WHERE Title NOT IN ('') AND DuringCOVIDVotes > 0
                                        ORDER BY DuringCOVIDVotes DESC, Votes DESC
                                        LIMIT 20
                                        """, conn)

conn.close()

## Bartosz Kosieradzki
#### 2023-01-07

## Introduction

This report provides a detailed analysis of the solutions to assignment number 2 in course Data Processing in R and Python. The report includes description of the solutions and comparison of correctness. Each query from the assignment is discussed in detail in each subsection in the report. For comparing results there is used .equals() function.

In [12]:
solution_2(Users,Posts).equals(res2)

True

In [13]:
solution_5(Posts, Votes).equals(res5)

KeyboardInterrupt: 

In [ ]:
solution_5(Posts, Votes)

In [ ]:
res5